In [1]:
import sys

sys.path.insert(0, '/home/storage2/hans/jax_reco/python')

import numpy as np
import jax.numpy as jnp
import jax

from trafos import transform_network_outputs, transform_dimensions, transform_dimensions_vec

In [2]:
dist = 25
z = -210
rho = 0.0
zenith = 90.0
azimuth = 0.0

x = transform_dimensions(dist, rho, z, zenith, azimuth)

In [3]:
np.random.seed(2)

batch_size = 500

x = np.array(x)
xx = x[np.newaxis, :]
xx = np.repeat(xx, batch_size, axis=0)
y = np.random.normal(0.025, 0.001, 500)
xx[:, 0] = y

zz = transform_dimensions_vec(xx)

In [4]:
def transform_dimensions_new(x):
    km_scale = 1000
    dist = x[0]
    rho = x[1]
    z = x[2]
    zenith = x[3]
    azimuith = x[4]
    
    x0 = dist / km_scale
    x1 = jnp.cos(rho)
    x2 = jnp.sin(rho)
    x3 = z / km_scale

    z = jnp.cos(jnp.deg2rad(zenith))
    x = jnp.sin(jnp.deg2rad(zenith)) * jnp.cos(jnp.deg2rad(azimuth))
    y = jnp.sin(jnp.deg2rad(zenith)) * jnp.sin(jnp.deg2rad(azimuth))
    return jnp.array([x0, x1, x2, x3, z, x, y])

In [5]:
x = jnp.array([dist, z, rho, zenith, azimuth])

In [6]:
print(transform_dimensions_new(x))

[ 2.500000e-02 -8.838775e-01 -4.677185e-01  0.000000e+00 -4.371139e-08
  1.000000e+00  0.000000e+00]


In [7]:
transform_v = jax.vmap(transform_dimensions_new, 0, 0)

In [8]:
yy = transform_v(jnp.array(xx))

In [9]:
print(yy.shape)

(500, 7)


In [10]:
print(zz.shape)

(500, 7)


In [12]:
print(yy[1], zz[1])

[ 2.4943734e-05  5.4030228e-01  8.4147102e-01  0.0000000e+00
  9.9999326e-01 -3.6651834e-03 -0.0000000e+00] [ 2.4943734e-05  5.4030228e-01  8.4147102e-01  0.0000000e+00
  9.9999326e-01 -3.6651834e-03  2.7961964e-12]


In [13]:
from trafos import transform_inputs_v

In [14]:
transform_inputs_v(jnp.array(xx))

Array([[ 2.4583243e-05,  5.4030228e-01,  8.4147102e-01, ...,
         9.9999326e-01, -3.6651834e-03,  2.7961964e-12],
       [ 2.4943734e-05,  5.4030228e-01,  8.4147102e-01, ...,
         9.9999326e-01, -3.6651834e-03,  2.7961964e-12],
       [ 2.2863804e-05,  5.4030228e-01,  8.4147102e-01, ...,
         9.9999326e-01, -3.6651834e-03,  2.7961964e-12],
       ...,
       [ 2.5236042e-05,  5.4030228e-01,  8.4147102e-01, ...,
         9.9999326e-01, -3.6651834e-03,  2.7961964e-12],
       [ 2.5494037e-05,  5.4030228e-01,  8.4147102e-01, ...,
         9.9999326e-01, -3.6651834e-03,  2.7961964e-12],
       [ 2.6546279e-05,  5.4030228e-01,  8.4147102e-01, ...,
         9.9999326e-01, -3.6651834e-03,  2.7961964e-12]], dtype=float32)